# 1. Cargar informacion

In [1]:
import pandas as pd

In [2]:
path = r'C:\Users\Ernes\OneDrive\Documents\Portafolio\north-america-retail-supply-chain-sales\north-america-retail-supply-chain-sales.xlsx'
df = pd.read_excel(path)
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Retail Sales People,Product ID,Category,Sub-Category,Product Name,Returned,Sales,Quantity,Discount,Profit
0,646,CA-2017-126221,2017-12-30,2018-05-01,Standard Class,CC-12430,Chuck Clark,Home Office,United States,Columbus,...,Kelly Williams,OFF-AP-10002457,Office Supplies,Appliances,Eureka The Boss Plus 12-Amp Hard Box Upright V...,Not,209.300,2,0.0,125.5800
1,5132,CA-2017-146626,2017-12-29,2018-05-01,Standard Class,BP-11185,Ben Peterman,Corporate,United States,Anaheim,...,Anna Andreadi,FUR-FU-10002501,Furniture,Furnishings,Nu-Dell Executive Frame,Not,101.120,8,0.0,60.6720
2,571,CA-2017-164826,2017-12-28,2018-04-01,Standard Class,JF-15415,Jennifer Ferguson,Consumer,United States,New York City,...,Chuck Magee,OFF-LA-10001297,Office Supplies,Labels,Avery 473,Not,72.450,7,0.0,43.4700
3,572,CA-2017-164826,2017-12-28,2018-04-01,Standard Class,JF-15415,Jennifer Ferguson,Consumer,United States,New York City,...,Chuck Magee,OFF-FA-10000585,Office Supplies,Fasteners,OIC Bulk Pack Metal Binder Clips,Not,13.960,4,0.0,8.3760
4,573,CA-2017-164826,2017-12-28,2018-04-01,Standard Class,JF-15415,Jennifer Ferguson,Consumer,United States,New York City,...,Chuck Magee,OFF-BI-10001922,Office Supplies,Binders,Storex Dura Pro Binders,Not,33.264,7,0.2,19.9584


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Row ID               9994 non-null   int64         
 1   Order ID             9994 non-null   object        
 2   Order Date           9994 non-null   datetime64[ns]
 3   Ship Date            9994 non-null   datetime64[ns]
 4   Ship Mode            9994 non-null   object        
 5   Customer ID          9994 non-null   object        
 6   Customer Name        9994 non-null   object        
 7   Segment              9994 non-null   object        
 8   Country              9994 non-null   object        
 9   City                 9994 non-null   object        
 10  State                9994 non-null   object        
 11  Postal Code          9994 non-null   int64         
 12  Region               9994 non-null   object        
 13  Retail Sales People  9994 non-nul

# 2. Limpieza

## 2.1 Valores duplicados

In [4]:
df.duplicated().any()

False

## 2.2 Valores nulos.

In [5]:
df.isnull().sum()

Row ID                 0
Order ID               0
Order Date             0
Ship Date              0
Ship Mode              0
Customer ID            0
Customer Name          0
Segment                0
Country                0
City                   0
State                  0
Postal Code            0
Region                 0
Retail Sales People    0
Product ID             0
Category               0
Sub-Category           0
Product Name           0
Returned               0
Sales                  0
Quantity               0
Discount               7
Profit                 0
dtype: int64

Existen 7 valores nulos en la columna discount, esto quiere decir que esos siete registros no tuvieron descuentos por lo que se llenaran con el valor ``0``.

In [6]:
df.fillna({'Discount':0}, inplace=True)

In [7]:
#Columnas de interés como tablas de Dimensión
columnas = [

    "Order ID",
    "Ship Mode",
    "Customer ID",
    "Customer Name",
    "Segment",
    "Country",
    "City",
    "State",
    "Postal Code",
    "Region",
    "Retail Sales People",
    "Product ID",
    "Category",
    "Sub-Category",
    "Product Name"
]


In [8]:
valores_unicos = {}
for col in columnas:
    valores_unicos[col] = df[col].nunique()

In [9]:
valores_unicos

{'Order ID': 5009,
 'Ship Mode': 4,
 'Customer ID': 793,
 'Customer Name': 793,
 'Segment': 3,
 'Country': 1,
 'City': 531,
 'State': 49,
 'Postal Code': 631,
 'Region': 4,
 'Retail Sales People': 4,
 'Product ID': 1862,
 'Category': 3,
 'Sub-Category': 17,
 'Product Name': 1850}

Crearemos un modelo de datos con las siguientes tablas de dimension.

# 3. Modelo de datos.

Crearemos nuestro modelo de datos según la Normalización de bases de datos.

In [10]:
def map_column_from_dimension(df, source_col, dim_df, dim_key_col, dim_id_col, new_col_name):
    """
    Reemplaza valores de df[source_col] con IDs desde una tabla de dimensión.
    
    Parámetros:
    - df: DataFrame origen (tabla de hechos)
    - source_col: columna en df a mapear
    - dim_df: DataFrame de dimensión
    - dim_key_col: columna clave en la tabla de dimensión (ej: 'Seller Name')
    - dim_id_col: columna con el ID a traer (ej: 'IDSeller')
    - new_col_name: nombre de la nueva columna en df
    """
    mapping_dict = dim_df.set_index(dim_key_col)[dim_id_col]
    df[new_col_name] = df[source_col].map(mapping_dict)

    return df

## 3.1 Tablas de dimension.

In [11]:
#col: 'Ship Mode'
DimShip = pd.DataFrame({
    'IDShip': list(range(1, 5)),
    'Ship Mode': list(df['Ship Mode'].unique())
})

In [12]:
# col : 'Retail Sales People' 
DimSellers = pd.DataFrame({
    'IDSeller': list(range(1,5)),
    'Seller Name': list(df['Retail Sales People'].unique())
}) 

In [13]:
# col : 'Segment' 
DimSegment = pd.DataFrame({
    'IDSegment' : list(range(1,4)),
    'Segment': list(df['Segment'].unique())
})

In [14]:
# col: 'Customer ID'
DimCustomers =  df[['Customer ID', 'Customer Name']].copy()
DimCustomers = DimCustomers.drop_duplicates()

In [15]:
# cols : Info sobre la region
DimRegion = df[['Postal Code', 'Country', 'City', 'State', 'Region']].copy()
DimRegion = DimRegion.drop_duplicates(subset=['Postal Code'])

In [16]:

DimProducts = df[['Product ID','Product Name', 'Category', 'Sub-Category']].copy()
DimProducts = DimProducts.drop_duplicates(subset=['Product ID'])

## 3.2 Tablas de Hechos.

In [17]:
# Mapear Seller
df = map_column_from_dimension(df, 'Retail Sales People', DimSellers, 'Seller Name', 'IDSeller', 'IDSeller')

# Mapear Ship Mode
df = map_column_from_dimension(df, 'Ship Mode', DimShip, 'Ship Mode', 'IDShip', 'IDShip')

# Mapear Segment
df =map_column_from_dimension(df,'Segment',DimSegment,'Segment','IDSegment','IDSegment')

In [18]:
FactTransaction = df[['Order ID','Customer ID','IDSeller','Order Date','Ship Date','Postal Code','IDShip','IDSegment','Returned']].copy()
FactTransaction = FactTransaction.drop_duplicates()

FactSales = df[['Order ID','Product ID','Sales','Quantity','Discount','Profit']].copy()
FactSales = FactSales.drop_duplicates()

# 4. Exportar informacion

## 4.1 Tablas de Hecho

In [19]:
FactTransaction.to_csv('FactTransaction.csv',index=False)
FactSales.to_csv('FactSales.csv',index=False)

## 4.2 Tablas de Dimensión

In [ ]:
DimShip.to_csv('DimShip.csv',index=False)
DimSellers.to_csv('DimSellers.csv',index=False)
DimSegment.to_csv('DimSegment.csv',index=False)
DimCustomers.to_csv('DimCustomers.csv',index=False)
DimRegion.to_csv('DimRegion.csv',index=False)
DimProducts.to_csv('DimProducts.csv',index=False)